<a href="https://colab.research.google.com/github/KevinHern/SemOpLabs/blob/master/Lab6/Laboratorio6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only works in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import datetime, os
import numpy as np
import pandas as pd

# Google Drive

## Authorization

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dataset

## Useful Functions

In [0]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()

    print("\nShape dimensions:\n")
    for name in filenames:
        data = load_file(prefix + name)
        print(np.array(loaded).shape)
        loaded.append(data)
      
     
    # stack group so that features are the 3rd dimension
    print(np.array(loaded).shape)
    loaded = np.dstack(loaded)
    print(np.array(loaded).shape)
    return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'UCI HAR Dataset/')
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'UCI HAR Dataset/')
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    print("\nOne Hot Encoding: \n",trainy)
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print("\n",trainy)
    return trainX, trainy, testX, testy

## Downloading

In [18]:
dataset_path = "/content/drive/My Drive/SP1_2020/lab6/"

# run this only once to save dataset to drive
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/HAR_Smartphones.zip -P "/content/drive/My Drive/SP1_2020/lab6/"

--2020-03-02 02:36:00--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/HAR_Smartphones.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60999314 (58M) [application/zip]
Saving to: ‘/content/drive/My Drive/SP1_2020/lab6/HAR_Smartphones.zip.1’

HAR_Smartphones.zip 100%[===================>]  58.17M  54.7MB/s    in 1.1s    

2020-03-02 02:36:02 (54.7 MB/s) - ‘/content/drive/My Drive/SP1_2020/lab6/HAR_Smartphones.zip.1’ saved [60999314/60999314]



In [19]:
!unzip -q "/content/drive/My Drive/SP1_2020/lab6/HAR_Smartphones.zip" -d "/content/drive/My Drive/SP1_2020/lab6/"

replace /content/drive/My Drive/SP1_2020/lab6/UCI HAR Dataset/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/drive/My Drive/SP1_2020/lab6/__MACOSX/UCI HAR Dataset/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/drive/My Drive/SP1_2020/lab6/UCI HAR Dataset/activity_labels.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/drive/My Drive/SP1_2020/lab6/__MACOSX/UCI HAR Dataset/._activity_labels.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/drive/My Drive/SP1_2020/lab6/UCI HAR Dataset/features.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/drive/My Drive/SP1_2020/lab6/__MACOSX/UCI HAR Dataset/._features.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/drive/My Drive/SP1_2020/lab6/UCI HAR Dataset/features_info.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace /content/drive/My Drive/SP1_2020/lab6/UCI HAR Dataset/features_info.txt? [y]es, [n]o, [A]ll, [N]one, [r]e

## Construction

In [24]:
trainX, trainy, testX, testy = load_dataset(dataset_path)


Shape dimensions:

(0,)
(1, 7352, 128)
(2, 7352, 128)
(3, 7352, 128)
(4, 7352, 128)
(5, 7352, 128)
(6, 7352, 128)
(7, 7352, 128)
(8, 7352, 128)
(9, 7352, 128)
(7352, 128, 9)

Shape dimensions:

(0,)
(1, 2947, 128)
(2, 2947, 128)
(3, 2947, 128)
(4, 2947, 128)
(5, 2947, 128)
(6, 2947, 128)
(7, 2947, 128)
(8, 2947, 128)
(9, 2947, 128)
(2947, 128, 9)

One Hot Encoding: 
 [[4]
 [4]
 [4]
 ...
 [1]
 [1]
 [1]]

 [[0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 ...
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]


# Model

## Imports

In [51]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.utils import to_categorical

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Callbacks

In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
# checkpoint
filepath = dataset_path + "output.best.h5"
checkpoint_callback = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min',save_freq='epoch')

# tensorboard
#logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
#tensorboard_callback = TensorBoard(logdir, histogram_freq=1,profile_batch=0)

# callbacks
#callbacks_list = [checkpoint_callback,tensorboard_callback]
callbacks_list = [checkpoint_callback]

## Definition

In [0]:
model = Sequential()

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    global model
    verbose, epochs, batch_size = 1, 10, 32
    samples, n_timesteps, n_features, n_outputs = trainX.shape[0], trainX.shape[1], trainX.shape[2], trainy.shape[1]
    
    model.add(Bidirectional(LSTM(64, activation='relu', input_shape=(n_timesteps, n_features))))
    #model.add(Bidirectional(LSTM(16)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose,callbacks=callbacks_list)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

## Training

In [66]:
score = evaluate_model(trainX, trainy, testX, testy)
score = score * 100.0
print('>Accuracy: %.3f' % (score))

Train on 7352 samples
Epoch 1/10
7328/7352 [============================>.] - ETA: 0s - loss: 52081140.9090 - accuracy: 0.6164
Epoch 00001: loss improved from inf to 51911126.30662, saving model to /content/drive/My Drive/SP1_2020/lab6/output.best.h5
7352/7352 [==============================] - 29s 4ms/sample - loss: 51911126.3066 - accuracy: 0.6163
Epoch 2/10
7328/7352 [============================>.] - ETA: 0s - loss: 0.8466 - accuracy: 0.6921
Epoch 00002: loss improved from 51911126.30662 to 0.84664, saving model to /content/drive/My Drive/SP1_2020/lab6/output.best.h5
7352/7352 [==============================] - 27s 4ms/sample - loss: 0.8466 - accuracy: 0.6921
Epoch 3/10
7328/7352 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.1691
Epoch 00003: loss did not improve from 0.84664
7352/7352 [==============================] - 27s 4ms/sample - loss: nan - accuracy: 0.1692
Epoch 4/10
7328/7352 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.1668


## Predict

In [67]:
t = testX[5]
t = t.reshape(1,128,9)


print("\nclass:",model.predict_classes(t))
print("\nprobs:",model.predict_proba(t))

#print(np.argmax(result))



class: [0]

probs: [[nan nan nan nan nan nan]]
